In [1]:
!pip install transformers accelerate torch pillow safetensors


In [4]:
from huggingface_hub import login
login()



In [2]:
from PIL import Image
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

MODEL_ID = "google/paligemma-3b-pt-224"

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
).eval()


Using device: cuda


preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [32]:
image_path = "/content/test image 3.jpg"   # change if your filename is different
prompt = "this building is "

image = Image.open(image_path).convert("RGB")


In [33]:
def ask_paligemma(image, prompt):
    inputs = processor(
        text=[prompt],
        images=[image],
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=120,      # <-- MORE TOKENS for full sentence
            do_sample=True,
            temperature=0.7,         # balanced creativity
            top_p=0.9,
            repetition_penalty=1.2,  # avoids repeating same word
            num_beams=3              # makes output more coherent
        )

    answer = processor.decode(out[0], skip_special_tokens=True)
    print(answer)


In [34]:
ask_paligemma(
    image,
    "Describe this building in one sentence:"
)


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


Describe this building in one sentence:
beautiful


In [35]:
ask_paligemma(
    image,
    "What kind of building is shown in this image?"
)


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


What kind of building is shown in this image?
house


In [36]:
ask_paligemma(
    image,
    "What materials does this building appear to be made of?"
)


You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


What materials does this building appear to be made of?
grass


In [37]:
ask_paligemma(
    image,
    "which country is this located?"
)

You are passing both `text` and `images` to `PaliGemmaProcessor`. The processor expects special image tokens in the text, as many tokens as there are images per each text. It is recommended to add `<image>` tokens in the very beginning of your text. For this call, we will infer how many images each text has and add special tokens.


which country is this located?
us
